# Quick Start Tutorial: Model Training

## Learning Objectives

In this tutorial you will learn:
1. How to design an observation set for your use case
2. How to materialize training data
3. How your ML training environment can consume training data

## Set up the prerequisites

Learning Objectives

In this section you will:
* start your local featurebyte server
* import libraries
* learn the about catalogs
* activate a pre-built catalog

In [1]:
# library imports
import pandas as pd
import numpy as np
import random

# load the featurebyte SDK
import featurebyte as fb

# start the local server, then wait for it to be healthy before proceeding
fb.playground()

12:24:41 | INFO     | Using configuration file at: /Users/jevonyeoh/.featurebyte/config.yaml
12:24:41 | WARNING  | No valid profile specified. Update config file or specify valid profile name with "use_profile".
12:24:41 | INFO     | (1/4) Starting featurebyte services
Container redis  Running
Container mongo-rs  Running
Container spark-thrift  Running
Container featurebyte-worker  Running
Container featurebyte-server  Running
Container mongo-rs  Waiting
Container redis  Waiting
Container mongo-rs  Waiting
Container redis  Healthy
Container mongo-rs  Healthy
Container mongo-rs  Healthy
12:24:43 | INFO     | (2/4) Creating local spark feature store
12:24:43 | INFO     | (3/4) Import datasets
12:24:50 | INFO     | Dataset grocery already exists, skipping import
12:24:50 | INFO     | Dataset healthcare already exists, skipping import
12:24:50 | INFO     | Dataset creditcard already exists, skipping import
12:24:50 | INFO     | (4/4) Playground environment started successfully. Ready to go

### Create a pre-built catalog for this tutorial, with the data, metadata, and features already set up

Note that creating a pre-built catalog is not a step you will do in real-life. This is a function specific to this quick-start tutorial to quickly skip over many of the preparatory steps and get you to a point where you can materialize features.

In a real-life project you would do data modeling, declaring the tables, entities, and the associated metadata. This would not be a frequent task, but forms the basis for best-practice feature engineering.

In [2]:
# get the functions to create a pre-built catalog
from prebuilt_catalogs import *

# create a new catalog for this tutorial
catalog = create_tutorial_catalog(PrebuiltCatalog.QuickStartModelTraining)

Cleaning up existing tutorial catalogs


12:25:23 | INFO     | Catalog activated: quick start model training 20230802:1213


Cleaning catalog: quick start model training 20230802:1213
  1 historical feature tables
  2 observation tables
Done! |████████████████████████████████████████| 100% in 9.2s (0.11%/s)         
Done! |████████████████████████████████████████| 100% in 9.2s (0.11%/s)         
Done! |████████████████████████████████████████| 100% in 9.2s (0.11%/s)         


12:25:52 | INFO     | Catalog activated: quick start model training 20230802:1225


Building a quick start catalog for model training named [quick start model training 20230802:1225]
Creating new catalog
Catalog created
Registering the source tables
Registering the entities
Tagging the entities to columns in the data tables
Populating the feature store with example features
Done! |████████████████████████████████████████| 100% in 6.1s (0.17%/s)         
Done! |████████████████████████████████████████| 100% in 6.1s (0.17%/s)         
Done! |████████████████████████████████████████| 100% in 6.1s (0.17%/s)         
Loading Feature(s) |████████████████████████████████████████| 1/1 [100%] in 0.1s
Done! |████████████████████████████████████████| 100% in 6.1s (0.17%/s)         
Done! |████████████████████████████████████████| 100% in 6.1s (0.17%/s)         
Done! |████████████████████████████████████████| 100% in 6.1s (0.17%/s)         
Done! |████████████████████████████████████████| 100% in 6.2s (0.16%/s)         
Done! |████████████████████████████████████████| 100% in 6.

### Example: Create views from tables in the Catalog

In [3]:
# create the views
grocery_customer_view = catalog.get_view("GROCERYCUSTOMER")
grocery_invoice_view = catalog.get_view("GROCERYINVOICE")
grocery_items_view = catalog.get_view("INVOICEITEMS")
grocery_product_view = catalog.get_view("GROCERYPRODUCT")

## Create an observation set for your use case

Learning Objectives

In this section you will learn:
* the purpose of observation sets
* the relationship between entities, point in time, and observation sets
* how to design an observation set suitable for training data

### Case Study: Predicting Customer Spend

Your chain of grocery stores wants to target market customers immediately after each purchase. As one step in this marketing campaign, they want to predict future customer spend in the 14 days after a purchase.

### Concept: Materialization

A feature in FeatureByte is defined by the logical plan for its computation. The act of computing the feature is known as Feature Materialization.

The materialization of features is made on demand to fulfill historical requests, whereas for prediction purposes, feature values are generated through a batch process called a "Feature Job". The Feature Job is scheduled based on the defined settings associated with each feature.

### Concept: Observation set

An observation set combines entity key values and historical points-in-time, for which you wish to materialize feature values.

The observation set can be a pandas DataFrame or an ObservationTable object representing an observation set in the feature store. An accepted serving name must be used for the column containing the entity values. The column containing points-in-time must be labelled "POINT-IN-TIME" and the point-in-time timestamps should be in UTC.

### Concept: Point in time

A point-in-time for a feature refers to a specific moment in the past with which the feature's values are associated.

It is a crucial aspect of historical feature serving, which allows machine learning models to make predictions based on historical data. By providing a point-in-time, a feature can be used to train and test models on past data, enabling them to make accurate predictions for similar situations in the future.

### Case Study: Predicting Customer Spend

Your chain of grocery stores wants to target market customers immediately after each purchase. As one step in this marketing campaign, they want to predict future customer spend in the 14 days after a purchase.

In [4]:
# get the feature list for the target feature
customer_target = catalog.get_target("next_customer_sales_14d")

# display details about the target feature
display(customer_target.info())

{
  'id': '64c9db3dafc5b31039def2a2',
  'target_name': 'next_customer_sales_14d',
  'entities': [
    {
      'name': 'grocerycustomer',
      'serving_names': [
        'GROCERYCUSTOMERGUID'
      ],
      'catalog_name': 'quick start model training 20230802:1225'
    }
  ],
  'window': '14d',
  'has_recipe': True,
  'created_at': '2023-08-02T04:27:41.163000',
  'updated_at': None,
  'input_data': {
    'main_data': {
      'name': 'GROCERYCUSTOMER',
      'data_type': 'scd_table'
    },
    'other_data': None
  },
  'metadata': {
    'input_columns': {
      'Input0': {
        'data': 'GROCERYINVOICE',
        'column_name': 'Timestamp',
        'semantic': 'event_timestamp'
      },
      'Input1': {
        'data': 'GROCERYINVOICE',
        'column_name': 'Amount',
        'semantic': None
      }
    },
    'derived_columns': {},
    'aggregations': {
      'F0': {
        'name': 'next_customer_sales_14d',
        'column': 'Input1',
        'function': 'sum',
        'keys': [
          'GroceryCustomerGuid'
        ],
        'window': '14d',
        'category': None,
        'filter': False
      }
    },
    'post_aggregation': None
  },
  'namespace_description': None,
  'description': None
}

In [5]:
# create a large observation table from a view

# filter the view to exclude points in time that won't have data for historical windows
filter = (grocery_invoice_view["Timestamp"] >= pd.to_datetime("2022-04-01")) & (
    grocery_invoice_view["Timestamp"] < pd.to_datetime("2023-04-01")
)
observation_set_view = grocery_invoice_view[filter].copy()

# create a new observation table
observation_table = observation_set_view.create_observation_table(
    name="10,000 Customers immediately after each purchase from May-22 to Mar-23",
    sample_rows=10000,
    columns=["Timestamp", "GroceryCustomerGuid"],
    columns_rename_mapping={
        "Timestamp": "POINT_IN_TIME",
        "GroceryCustomerGuid": "GROCERYCUSTOMERGUID",
    },
)

# if the observation table isn't too large, you can materialize it
display(observation_table.to_pandas())

Done! |████████████████████████████████████████| 100% in 21.8s (0.05%/s)        


,POINT_IN_TIME,GROCERYCUSTOMERGUID
0,2022-04-05 18:55:03,5c96089d-95f7-4a12-ab13-e082836253f1
1,2022-04-08 13:10:00,5c96089d-95f7-4a12-ab13-e082836253f1
2,2022-04-11 11:49:05,5c96089d-95f7-4a12-ab13-e082836253f1
3,2022-04-15 09:50:57,5c96089d-95f7-4a12-ab13-e082836253f1
4,2022-05-14 15:00:07,5c96089d-95f7-4a12-ab13-e082836253f1
...,...,...
9995,2022-04-24 09:47:55,493d4cbd-f86c-464c-b773-6ce88d416985
9996,2022-04-26 16:43:31,493d4cbd-f86c-464c-b773-6ce88d416985
9997,2022-05-03 14:17:36,493d4cbd-f86c-464c-b773-6ce88d416985
9998,2022-05-09 13:36:10,493d4cbd-f86c-464c-b773-6ce88d416985


## Materialize Training Data

Learning Objectives

In this section you will learn:
* how to create historical training data
* how to merge target and features

### Example: Get target values


In [6]:
# Materialize the target feature using get historical features
training_data_target_table = customer_target.compute_target_table(
    observation_table, observation_table_name="target_observation_table"
)

display(training_data_target_table.to_pandas())

Done! |████████████████████████████████████████| 100% in 28.1s (0.04%/s)        


,POINT_IN_TIME,GROCERYCUSTOMERGUID,next_customer_sales_14d
0,2022-11-18 12:24:01,9b1b8037-8506-4a54-981a-3b7e694a489f,81.43
1,2022-10-12 14:05:21,2b068f1d-d99b-4c2f-a737-46f619a76cc8,49.44
2,2022-10-22 21:08:34,871298ba-1659-47ba-a2de-d5e154ef4c5d,17.99
3,2022-09-19 17:37:03,42839dea-155f-46c3-b724-48b4c2328aed,NaN
4,2022-05-31 13:18:48,5c1e93ae-fa46-4a26-bb1d-6040603dad87,48.17
...,...,...,...
9995,2022-10-07 18:22:20,df23dfe3-fede-46aa-b283-422458c11767,51.33
9996,2022-08-16 14:34:40,f6a783f7-5091-46fa-8ebf-aa13ec868234,136.88
9997,2022-08-20 10:47:51,f6a783f7-5091-46fa-8ebf-aa13ec868234,63.26
9998,2022-12-30 08:00:14,ff38d86f-cd9a-4860-9b0a-eb387bfe0a10,40.55


### Example: Get historical values with target

In [7]:
# list the feature lists
display(catalog.list_feature_lists())

,id,name,num_feature,status,deployed,readiness_frac,online_frac,tables,entities,primary_entities,created_at
0,64c9db2fafc5b31039def29b,Features,8,DRAFT,False,1.0,0.0,"[GROCERYCUSTOMER, GROCERYINVOICE, INVOICEITEMS...","[grocerycustomer, frenchstate]",[grocerycustomer],2023-08-02T04:27:34.963000


In [8]:
# get the feature list
feature_list = catalog.get_feature_list("Features")

Loading Feature(s) |████████████████████████████████████████| 8/8 [100%] in 0.2s


In [9]:
# Compute the historical feature table by passing in the observation table that contains the target values
training_table_features = feature_list.compute_historical_feature_table(
    training_data_target_table,
    historical_feature_table_name="customer training table - invoices Apr-22 to Mar-23 - features only",
)

# display the training data
training_data = training_table_features.to_pandas()
display(training_data)

Done! |████████████████████████████████████████| 100% in 1:38.5 (0.01%/s)       


,POINT_IN_TIME,GROCERYCUSTOMERGUID,next_customer_sales_14d,StatePopulation,StateAvgInvoiceAmount_28d,StateMeanLatitude,StateMeanLongitude,CustomerInventoryStability_14d28d,CustomerStateSimilarity_28d,CustomerSpend_28d,CustomerAvgInvoiceAmount_28d
0,2022-04-01 08:43:25,352d1de1-4419-40e5-b2a5-6d6922384b05,6.79,183,18.021939,48.740582,2.237559,0.866025,0.367632,15.98,7.990000
1,2022-04-01 09:57:05,ed56f1f6-310d-4b7c-9f5b-554103282f15,94.06,3,15.970000,48.354199,-1.871965,1.000000,0.871330,35.84,35.840000
2,2022-04-01 10:41:29,8759ff7c-4cad-44e7-82dd-f89c925699be,15.66,14,15.516444,43.404298,3.330159,0.931614,0.578746,54.94,10.988000
3,2022-04-01 11:38:19,09fbee0c-521e-40ee-a2ff-8ed4187dcbc4,88.02,8,17.258158,48.789776,5.855939,0.877520,0.802827,268.19,26.819000
4,2022-04-01 12:20:01,b21ae11c-83cf-4146-832e-1163413a3295,21.73,5,8.032955,49.185500,-0.530407,0.968960,0.823614,94.11,2.940938
...,...,...,...,...,...,...,...,...,...,...,...
9995,2022-12-31 14:15:06,b429441a-8a9f-4d54-8aba-835be15192c4,2.99,181,19.304609,48.739038,2.242254,0.874818,0.550837,44.71,8.942000
9996,2022-12-31 17:04:34,c52ea584-e4e4-4edb-8ba4-99cecfdaa08d,30.19,8,18.215833,49.446984,2.414170,0.235170,0.800712,235.47,29.433750
9997,2022-12-31 17:09:22,e034e01c-50de-42f0-a879-82c093af5f49,NaN,181,19.159682,48.739038,2.242254,0.600000,0.713417,25.23,12.615000
9998,2022-12-31 18:06:00,b26ee24d-d02a-41c8-a107-9c4f084da61d,10.56,181,19.238506,48.739038,2.242254,0.991489,0.547564,84.78,14.130000


## Consuming training data

Learning Objectives

In this section you will learn:
* how to save a training file
* how to use a pandas data frame

### Example: Save the training data to a file

In [10]:
# save training data as a csv file
training_data.to_csv("training_data.csv", index=False)

In [11]:
# save the training file as a parquet file
training_data.to_parquet("training_data.parquet")

### Example: Training a scikit learn model

Note that you will need to install scikit learn https://scikit-learn.org/stable/install.html

In [12]:
# EDA on the training data
training_data.describe()

,next_customer_sales_14d,StatePopulation,StateAvgInvoiceAmount_28d,StateMeanLatitude,StateMeanLongitude,CustomerInventoryStability_14d28d,CustomerStateSimilarity_28d,CustomerSpend_28d,CustomerAvgInvoiceAmount_28d
count,8986.000000,10000.000000,9996.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,9625.000000
mean,73.534279,79.774600,18.223672,45.366231,3.248752,0.761324,0.592674,136.640595,18.253898
std,68.291312,75.686876,3.647413,9.621577,8.996799,0.296796,0.218917,123.759052,15.081509
min,0.000000,1.000000,3.887857,-12.713308,-50.017299,0.000000,0.000000,0.000000,0.620000
25%,23.472500,14.000000,16.583590,43.706807,2.237559,0.727249,0.483864,44.235000,8.425227
50%,52.815000,33.000000,17.824807,48.211446,2.241215,0.886785,0.631822,99.785000,14.637727
75%,102.007500,180.000000,20.125597,48.739485,5.054081,0.946294,0.746201,195.957500,23.325000
max,487.200000,183.000000,38.788889,50.669452,45.189819,1.000000,1.000000,837.360000,332.300000


In [13]:
# do any columns in the training data contain missing values?
training_data.isna().any()

POINT_IN_TIME                        False
GROCERYCUSTOMERGUID                  False
next_customer_sales_14d               True
StatePopulation                      False
StateAvgInvoiceAmount_28d             True
StateMeanLatitude                    False
StateMeanLongitude                   False
CustomerInventoryStability_14d28d    False
CustomerStateSimilarity_28d          False
CustomerSpend_28d                    False
CustomerAvgInvoiceAmount_28d          True
dtype: bool

In [14]:
! pip install scikit-learn

In [15]:
# use sklearn to train a random forest regression model on the training data
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    training_data.drop(columns=["GROCERYCUSTOMERGUID", "POINT_IN_TIME"]),
    training_data["next_customer_sales_14d"],
    test_size=0.2,
    random_state=42,
)

# train the model
model = HistGradientBoostingRegressor()
model.fit(X_train, y_train)

# get predictions
y_pred = model.predict(X_test)

# calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean squared error: ", mse)

# save the model
import joblib

joblib.dump(model, "model.pkl")

ValueError: Input y contains NaN.

## Next Steps

Now that you've completed the quick-start feature engineering tutorial, you can put your knowledge into practice or learn more:<br>
1. Learn more about materializing features via the "Deep Dive Materializing Features" tutorial
2. Put your knowledge into practice by creating features in the "credit card dataset feature engineering playground" or "healthcare dataset feature engineering playground" workspaces
3. Learn more about feature governance via the "Quick Start Feature Governance" tutorial